In [3]:
from datasets import load_dataset
import uuid
from llama_index.core.schema import TextNode
nodes = []
def _load_dataset(path="khanglt0004/vietnamese_legal_chunks"):
    dataset = load_dataset(path)
    for item in dataset['train']:
        new_node = TextNode(
            text=item['text'],
            id_=str(uuid.uuid5(uuid.NAMESPACE_DNS, str(item['id']))),
            metadata=item['metadata']
        )
        nodes.append(new_node)
    print("Đã tải dữ liệu các chunks, số lượng: ", len(nodes))
_load_dataset()

Đã tải dữ liệu các chunks, số lượng:  3245


In [5]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core.indices.vector_store import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="bkai-foundation-models/vietnamese-bi-encoder")
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")
vector_store = QdrantVectorStore(
    client=client, 
    collection_name="local_law_documents",
    enable_hybrid=True,
    fastembed_sparse_model="Qdrant/bm25",
    batch_size=20,
    )

storage_context = StorageContext.from_defaults(vector_store=vector_store)
# NOTE: reate Vector Database in Qdrant
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model = embed_model,
)

loaded_index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model= embed_model
)